In [1]:
%load_ext autoreload 
%autoreload 2
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
from causaldag import UndirectedGraph
import itertools
from sklearn import preprocessing
import pickle
from graphviz_plotting import plot_graphviz_style_graph
import os
from IPython.display import Image

%matplotlib inline
import numpy as np
from causaldag import unknown_target_igsp
import causaldag as cd
import random
from causaldag import gauss_invariance_suffstat, gauss_invariance_test, MemoizedInvarianceTester, gspo, gsp
from causaldag import partial_correlation_suffstat, partial_correlation_test, MemoizedCI_Tester

In [2]:
from causaldag import unknown_target_igsp
from causaldag import gauss_invariance_suffstat, gauss_invariance_test, MemoizedInvarianceTester
from causaldag import partial_correlation_suffstat, partial_correlation_test, MemoizedCI_Tester
from causaldag import UndirectedGraph
import causaldag as cd
import random
import numpy as np
import itertools
from joblib import Parallel, delayed
from sklearn.utils import safe_mask
from sklearn.utils.random import sample_without_replacement
from typing import Dict, Optional, Any, List, Set, Union
##this is modified from Belyaeva, A. et al. Causal network models of SARS-CoV-2 expression and aging to identify candidates for drug repurposing. Nat Commun 12, 1024 (2021).
##and Squires, C., Wang, Y. & Uhler, C. Permutation-Based Causal Structure Learning with Unknown Intervention Targets. Preprint at https://doi.org/10.48550/arXiv.1910.09007 (2020).
def run_unknown_target_igsp(
    X,
    Y,
    alpha,
    alpha2,
    nodes: set,
    setting_list):
    # obtain sufficient statistics (causaldag.utils.ci_tests)
    Y=[Y]
    obs_suffstat = partial_correlation_suffstat(X)
    invariance_suffstat = gauss_invariance_suffstat(X,Y)
    # define CI tester
    ci_tester = MemoizedCI_Tester(partial_correlation_test, obs_suffstat, alpha=alpha)
    invariance_tester = MemoizedInvarianceTester(gauss_invariance_test, invariance_suffstat, alpha=alpha2)
    # run GSP
    print(invariance_tester)
    est_dag, est_targets_list = unknown_target_igsp(setting_list, nodes, ci_tester, invariance_tester)
    # convert dag to adjacency matrix, here specifying that the columns are "source" axis, so edge from j->i
    est_cpdag, _ = est_dag.cpdag().to_amat(source_axis=1)

    return est_cpdag



def unknown_target_igsp_stability_selection(
    X, 
    Y,
    alpha_grid, 
    alpha2,
    nodes: set,
    setting_list,
    n_samples1,
    n_samples2,
    verbose: bool = False,
    sample_fraction: float = 0.6,
    n_bootstrap_iterations: int = 100,
    bootstrap_threshold: float = 0.5,
    n_jobs: int = 1,
    random_state: int = None):
    """
    X: array, shape = [n_samples, n_features]
        Dataset. 
    alpha_grid: array-like
        Parameters to iterate over.
    nodes:
        Labels of nodes in the graph.
    depth:
        Maximum depth in depth-first search. Use None for infinite search depth.
    nruns:
        Number of runs of the algorithm. Each run starts at a random permutation and the sparsest DAG from all
        runs is returned.
    verbose:
        Verbosity of logging messages.
    initial_undirected:
        Option to find the starting permutation by using the minimum degree algorithm on an undirected graph that is
        Markov to the data. You can provide the undirected graph yourself, use the default 'threshold' to do simple
        thresholding on the partial correlation matrix, or select 'None' to start at a random permutation.
    initial_permutations:
        A list of initial permutations with which to start the algorithm. This option is helpful when there is
        background knowledge on orders. This option is mutually exclusive with initial_undirected.
    fixed_orders:
        Tuples (i, j) where i is known to come before j.
    fixed_adjacencies:
        Tuples (i, j) where i and j are known to be adjacent.
    fixed_gaps:
        Tuples (i, j) where i and j are known to be non-adjacent.
    sample_fraction: float, default = 0.7
        The fraction of samples to be used in each bootstrap sample.
        Should be between 0 and 1. If 1, all samples are used.
    n_bootstrap_iterations: int, default = 100
        Number of bootstrap samples to create.
    bootstrap_threshold: float, default = 0.5
        Threshold defining the minimum cutoff value for the stability scores. Edges with stability scores above
        the bootstrap_threshold are kept as part of the difference-DAG.
    n_jobs: int, default = 1
        Number of jobs to run in parallel.
    random_state: int, default = None
        Seed used by the random number generator.
    Returns
    -------
    adjacency_matrix: array, shape  = [n_features, n_features]
        Estimated CPDAG. Undirected edges are represented by assigning 1 in both directions, i.e. adjacency_matrix[i,j] = 1
        and adjacency_matrix[j,i] = 1. Otherwise for oriented edges, only adjacency_matrix[i,j] = 1 is assigned. 
        Assignment of 0 in the adjacency matrix represents no edge.
    stability_scores: array, shape = [n_params, n_features, n_features]
        Stability score of each edge for each parameter value in alpha_grid.
    References
    ----------
        [1] Meinshausen, N. and Buhlmann, P. (2010). Stability selection.
           Journal of the Royal Statistical Society: Series B (Statistical Methodology), 72(4), pp.417-473.
    """
    n_variables = len(nodes)
    n_params = len(alpha_grid)
    hyperparams = alpha_grid
    stability_scores = np.zeros((n_params, n_variables, n_variables))

    for idx, param in enumerate(hyperparams):
        print(param)
        if verbose > 0:
            print(
                "Fitting estimator for alpha = %.5f with %d bootstrap iterations" %
                (param, n_bootstrap_iterations))

        bootstrap_samples1 = bootstrap_generator(n_bootstrap_iterations, sample_fraction,
                                                 n_samples1)
        bootstrap_samples2 = bootstrap_generator(n_bootstrap_iterations, sample_fraction,
                                                 n_samples2)
        print(X.shape)
        bootstrap_results = Parallel(n_jobs, verbose=verbose
                                     )(delayed(run_unknown_target_igsp)(X[safe_mask(X, subsample1), :],
                                                    Y[safe_mask(Y, subsample2), :],
                                                    alpha=param,
                                                    alpha2=param,
                                                    nodes=nodes,
                                                    setting_list=setting_list)
                                       for subsample1,subsample2 in zip(bootstrap_samples1,bootstrap_samples2))

        # calculate stability scores
        stability_scores[idx] = np.array(bootstrap_results).mean(axis=0)

    return stability_scores


def bootstrap_generator(n_bootstrap_iterations, sample_fraction, n_samples, random_state=None):
    """Generates bootstrap samples from dataset."""
    n_subsamples = np.floor(sample_fraction * n_samples).astype(int)
    subsamples = []
    for _ in range(n_bootstrap_iterations):
        subsample = sample_without_replacement(n_samples, n_subsamples, random_state=None)
        subsamples.append(subsample)
    return subsamples

In [ ]:
path_in = '/home/gridsan/jyun/dci'
df_original = pd.read_csv('/home/gridsan/jyun/dci/bd21d_12_bd21d.csv', index_col=0)
df_original.shape
df_original.shape[0]
path_in = '/home/gridsan/jyun/dci'
df_original1 = pd.read_csv('/home/gridsan/jyun/dci/bd21d_12_bd21c.csv', index_col=0)
df_original1.shape[0]

df_original2 = pd.read_csv('/home/gridsan/jyun/dci/bd21d_12_bd31d.csv', index_col=0)
df_original2.shape
df_original3 = pd.read_csv('/home/gridsan/jyun/dci/bd21d_12_bd31c.csv', index_col=0)

df_original3.shape
df_original = pd.concat([df_original])
df_original1 = pd.concat([df_original1,df_original2,df_original3])
#df_original1=df_original1.iloc[:92 , :]
#df_original=df_original.iloc[: 92, :]



#df_original = pd.concat([df_original])
#df_original1 = pd.concat([df_original1,df_original2,df_original3])

df_original1.shape
df_original.shape
#df_original=df_original.iloc[:45 , :49]
#df_original=pd.concat([df_original.iloc[:45, 0], df_original.iloc[:45, 15:49]], axis=1)
#df_original=df_original.iloc[:46 , :49]
X=df_original
Y=df_original1
nsamples, nnodes = df_original.shape
nsamples
nnodes
nodes = set(range(nnodes))
nodes
alpha_grid = [0.0001, 0.001, 0.01, 0.05,0.1]
num_targets = 1
num_settings = 1
nodes = set(range(nnodes))
targets_list = [random.sample(nodes, num_targets) for _ in range(num_settings)]
setting_list = [dict(known_interventions=[]) for _ in targets_list]
print(setting_list)
alpha2=0.0001
X=X.to_numpy()
Y=Y.to_numpy()
df_original.transpose()
stability_scores = unknown_target_igsp_stability_selection(X, Y, alpha_grid, alpha2, nodes, setting_list, n_samples1=df_original.shape[0],n_samples2=df_original1.shape[0],n_bootstrap_iterations=1000, n_jobs=48, verbose=False,sample_fraction=0.85)
np.savetxt("/home/gridsan/jyun/dci/adjacency_matrix_bd21d_test_corrected.csv",pd.DataFrame(stability_scores.max(axis=0)), delimiter=",")